<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
import os
import re
import subprocess
from pathlib import Path
from google.colab import drive
import threading

# Mount Google Drive
print("\033[1;32;1m\n --> Mounting Google Drive...\033[0;37;0m\n")
drive.mount('/content/drive')

# Set up directories in Google Drive
DRIVE_PATH = '/content/drive/MyDrive/text-generation-webui'
os.makedirs(DRIVE_PATH, exist_ok=True)

os.environ.pop('PYTHONPATH', None)
os.environ.pop('MPLBACKEND', None)

# Check if installation exists in Drive, if not install there
if not Path(f'{DRIVE_PATH}/.git').exists():
  print("\033[1;32;1m\n --> Installing the web UI in Google Drive. This will take a while, but only needs to be done once.\033[0;37;0m\n")

  %cd /content/drive/MyDrive
  !git clone https://github.com/oobabooga/text-generation-webui
  %cd text-generation-webui

  # Install the project in an isolated environment
  !GPU_CHOICE=A \
  LAUNCH_AFTER_INSTALL=FALSE \
  INSTALL_EXTENSIONS=FALSE \
  ./start_linux.sh
else:
  print("\033[1;32;1m\n --> Found existing installation in Google Drive. Skipping installation.\033[0;37;0m\n")
  %cd {DRIVE_PATH}

# Add execute permissions to start_linux.sh
!chmod +x ./start_linux.sh

# Parameters
model_url = "https://huggingface.co/turboderp/gemma-2-9b-it-exl2" #@param {type:"string"}
branch = "8.0bpw" #@param {type:"string"}
command_line_flags = "--n-gpu-layers 128 --load-in-4bit --use_double_quant --no_flash_attn" #@param {type:"string"}
api = True #@param {type:"boolean"}

if api:
  for param in ['--api', '--public-api']:
    if param not in command_line_flags:
      command_line_flags += f" {param}"

model_url = model_url.strip()
if model_url != "":
    if not model_url.startswith('http'):
        model_url = 'https://huggingface.co/' + model_url

    # Download the model to Drive (will skip if already exists)
    url_parts = model_url.strip('/').strip().split('/')
    output_folder = f"{url_parts[-2]}_{url_parts[-1]}"
    branch = branch.strip('"\' ')

    model_path = f"{DRIVE_PATH}/models/{output_folder}"

    if Path(model_path).exists():
        print(f"\033[1;32;1m\n --> Model already exists in Drive: {output_folder}\033[0;37;0m\n")
    else:
        print(f"\033[1;32;1m\n --> Downloading model to Drive: {output_folder}\033[0;37;0m\n")
        if branch.strip() not in ['', 'main']:
            output_folder += f"_{branch}"
            !python download-model.py {model_url} --branch {branch}
        else:
            !python download-model.py {model_url}
else:
    output_folder = ""

# Start the web UI and capture URLs
cmd = f"./start_linux.sh {command_line_flags} --share"
if output_folder != "":
    cmd += f" --model {output_folder}"

print("\n\033[1;35;1m" + "="*80)
print("STARTING WEB UI - WATCH FOR URLS BELOW")
print("="*80 + "\033[0;37;0m\n")

# Run the command
process = subprocess.Popen(
    cmd,
    shell=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    bufsize=1
)

# Monitor output for URLs
local_url = None
public_url = None

for line in iter(process.stdout.readline, ''):
    print(line, end='')

    # Capture local URL
    if 'Running on local URL:' in line or 'http://127.0.0.1' in line or 'http://0.0.0.0' in line:
        match = re.search(r'(http://[\d\.]+:\d+)', line)
        if match and not local_url:
            local_url = match.group(1)

    # Capture public URL
    if 'Running on public URL:' in line or 'gradio.live' in line:
        match = re.search(r'(https://[a-z0-9]+\.gradio\.live)', line)
        if match and not public_url:
            public_url = match.group(1)

    # Display URLs prominently when both are found
    if local_url and public_url:
        print("\n" + "\033[1;32;1m" + "="*80)
        print("🎉 WEB UI IS READY!")
        print("="*80)
        print(f"\n📍 LOCAL URL:  {local_url}")
        print(f"🌐 PUBLIC URL: {public_url}")
        print("\n" + "="*80 + "\033[0;37;0m\n")
        local_url = None  # Reset to avoid printing multiple times
        public_url = None

process.wait()